# Running tasks

Let's start by initializing nornir.

In [1]:
from nornir import InitNornir
nr = InitNornir(
    inventory={
        "options": {
            "host_file": "4_tasks/inventory/hosts.yaml",
            "group_file": "4_tasks/inventory/groups.yaml",
            "defaults_file": "4_tasks/inventory/defaults.yaml",
        }
    }
)

## Running a task

In [2]:
from nornir.plugins.tasks.networking import napalm_get

# we filter the hosts (optional, this would be the equivalent of "role")
spine_bma = nr.filter(role="spine", site="bma")

# we run the task, saving the result in the variable `r`
r = spine_bma.run(name="Get facts",
                  task=napalm_get,
                  getters=["facts"])

You can use the helper function `print_result` to print the result on the screen:

In [3]:
from nornir.plugins.functions.text import print_result

print_result(r)

Get facts***********************************************************************
* spine00.bma ** changed : False ***********************************************
vvvv Get facts ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': ['Ethernet1', 'Ethernet2', 'Management1'],
             'model': 'vEOS',
             'os_version': '4.20.1F-6820520.4201F',
             'serial_number': '',
             'uptime': 317919,
             'vendor': 'Arista'}}
^^^^ END Get facts ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* spine01.bma ** changed : False ***********************************************
vvvv Get facts ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'vsrx',
             'hostname': 'vsrx',
             'interface_list': ['ge-0/0/0', 'ge-0/0/1', 'vlan'],
             'model': 'FIREFLY-PERIMETER',
       

You can also work with the result programmatically:

In [4]:
# running a task will return a dict-like object
# where the keys are the hosts' names and the values
# a list of results (more on why a list later)
for host, task_results in r.items():
    print(f"{host}: {task_results}")

spine00.bma: MultiResult: [Result: "Get facts"]
spine01.bma: MultiResult: [Result: "Get facts"]


In [5]:
for host, task_results in r.items():
    my_task = task_results[0]         # the only task we ran
    my_task_result = my_task.result   # this should contain the exact output from napalm's `get_facts`
    print(f"{host}: {my_task_result['facts']['os_version']}")

spine00.bma: 4.20.1F-6820520.4201F
spine01.bma: 12.1X47-D20.7


## Running multiple tasks: a simple example

In [6]:
from nornir.plugins.tasks.networking import napalm_cli


# we group the tasks
def a_bunch_of_tasks(task):
    task.run(name="Get facts",
             task=napalm_get,
             getters=["facts"])
    task.run(name="Run CLI command",
             task=napalm_cli,
             commands=["show version"])
    return "I am done!!!"

    
r = spine_bma.run(name="Running a bunch of tasks",
                  task=a_bunch_of_tasks)

In [7]:
print_result(r)

Running a bunch of tasks********************************************************
* spine00.bma ** changed : False ***********************************************
vvvv Running a bunch of tasks ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I am done!!!
---- Get facts ** changed : False ---------------------------------------------- INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': ['Ethernet1', 'Ethernet2', 'Management1'],
             'model': 'vEOS',
             'os_version': '4.20.1F-6820520.4201F',
             'serial_number': '',
             'uptime': 317919,
             'vendor': 'Arista'}}
---- Run CLI command ** changed : False ---------------------------------------- INFO
{ 'show version': 'Arista vEOS\n'
                  'Hardware version:\n'
                  'Serial number:\n'
                  'System MAC address:  0800.273e.22fd\n'
                  '\n'
                  'Software image version: 4.20.

In [8]:
for host, task_results in r.items():
    grouped_result = task_results[0].result
    get_facts_result = task_results[1].result
    cli_result = task_results[2].result
    print(f"{host}: {get_facts_result['facts']['os_version']}")
    print(f"{cli_result['show version']}")
    print(f"---> {grouped_result}")
    print("=================")

spine00.bma: 4.20.1F-6820520.4201F
Arista vEOS
Hardware version:
Serial number:
System MAC address:  0800.273e.22fd

Software image version: 4.20.1F
Architecture:           i386
Internal build version: 4.20.1F-6820520.4201F
Internal build ID:
790a11e8-5aaf-4be7-a11a-e61795d05b91

Uptime:                 1 hour and 16 minutes
Total memory:           2017324 kB
Free memory:            1237124 kB

---> I am done!!!
spine01.bma: 12.1X47-D20.7
Hostname: vsrx
Model: firefly-perimeter
JUNOS Software Release [12.1X47-D20.7]

---> I am done!!!
